If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [147]:
#! pip install datasets transformers seqeval

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [148]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [149]:
# !apt install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [150]:
import transformers

print(transformers.__version__)

4.25.1


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/token-classification).

# Fine-tuning a model on a token classification task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a token classification task, which is the task of predicting a label for each token.

![Widget inference representing the NER task](images/token_classification.png)

The most common token classification tasks are:

- NER (Named-entity recognition) Classify the entities in the text (person, organization, location...).
- POS (Part-of-speech tagging) Grammatically classify the tokens (noun, verb, adjective...)
- Chunk (Chunking) Grammatically classify the tokens and group them into "chunks" that go together

We will see how to easily load a dataset for these kinds of tasks and use the `0er` API to fine-tune a model on it.

This notebook is built to run on any token classification task, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [151]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "bert-base-cased"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [2]:
from datasets import load_dataset, load_metric

For our example here, we'll use the [CONLL 2003 dataset](https://www.aclweb.org/anthology/W03-0419.pdf). The notebook should work with any token classification dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

In [3]:
datasets = load_dataset("jnlpba")

KeyboardInterrupt: 

In [185]:
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 37094
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 7714
    })
})


In [155]:
#print(datasets["train"][:2])

In [156]:
#jnlpba = load_dataset('jnlpba', split=['train[:2%]', "validation[:2%]"])

In [157]:
#datasets = jnlpba

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the 0ing, validation and test set.

In [158]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 37094
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 7714
    })
})

We can see the 0ing, validation and test sets all have a column for the tokens (the input texts split into words) and one column of labels for each kind of task we introduced before.

To access an actual element, you need to select a split first, then give an index:

In [159]:
datasets["train"][0]

{'id': '1',
 'tokens': ['IL-2',
  'gene',
  'expression',
  'and',
  'NF-kappa',
  'B',
  'activation',
  'through',
  'CD28',
  'requires',
  'reactive',
  'oxygen',
  'production',
  'by',
  '5-lipoxygenase',
  '.'],
 'ner_tags': [1, 2, 0, 0, 9, 10, 0, 0, 9, 0, 0, 0, 0, 0, 9, 0]}

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored in the `features` of the dataset:

In [160]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-DNA', 'I-DNA', 'B-RNA', 'I-RNA', 'B-cell_line', 'I-cell_line', 'B-cell_type', 'I-cell_type', 'B-protein', 'I-protein'], id=None), length=-1, id=None)

So for the NER tags, 0 corresponds to 'O', 1 to 'B-PER' etc... On top of the 'O' (which means no special entity), there are four labels for NER here, each prefixed with 'B-' (for beginning) or 'I-' (for intermediate), that indicate if the token is the first one for the current group with the label or not:
- 'PER' for person
- 'ORG' for organization
- 'LOC' for location
- 'MISC' for miscellaneous

Since the labels are lists of `ClassLabel`, the actual names of the labels are nested in the `feature` attribute of the object above:

In [161]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O',
 'B-DNA',
 'I-DNA',
 'B-RNA',
 'I-RNA',
 'B-cell_line',
 'I-cell_line',
 'B-cell_type',
 'I-cell_type',
 'B-protein',
 'I-protein']

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [162]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [163]:
show_random_elements(datasets["train"])

,id,tokens,ner_tags
0,7297,"[Here, we, report, that, in, erythroid, cells, ,, GATA-1, strongly, induces, the, expression, of, the, anti-apoptotic, protein, bcl-xL, ,, but, not, the, related, proteins, bcl-2, and, mcl-1, .]","[O, O, O, O, O, B-cell_type, I-cell_type, O, B-protein, O, O, O, O, O, O, B-protein, I-protein, B-protein, O, O, O, O, O, O, B-protein, O, B-protein, O]"
1,1640,"[Previously, ,, an, inducible, enhancer, between, nucleotides, -299, and, -228, that, contains, NF-kappa, B, and, CArG, motifs, was, identified, .]","[O, O, O, B-DNA, I-DNA, O, B-DNA, I-DNA, I-DNA, I-DNA, O, O, B-DNA, I-DNA, I-DNA, I-DNA, I-DNA, O, O, O]"
2,18025,"[In, this, study, ,, we, have, identified, a, kinase, in, cell, extracts, from, the, LPS-stimulated, human, monocytic, cell, line, ,, THP-1, ,, that, specifically, binds, and, phosphorylates, I, kappa, B, alpha, .]","[O, O, O, O, O, O, O, O, B-protein, O, O, O, O, O, B-cell_line, I-cell_line, I-cell_line, I-cell_line, I-cell_line, O, B-cell_line, O, O, O, O, O, O, B-protein, I-protein, I-protein, I-protein, O]"
3,16050,"[The, dimerization, domain, of, KBF1/p50, is, contained, between, amino, acids, 201, and, 367, .]","[O, B-protein, I-protein, O, B-protein, O, O, B-protein, I-protein, I-protein, I-protein, I-protein, I-protein, O]"
4,14629,"[A, human, putative, lymphocyte, G0/G1, switch, gene, homologous, to, a, rodent, gene, encoding, a, zinc-binding, potential, transcription, factor, .]","[O, B-DNA, I-DNA, I-DNA, I-DNA, I-DNA, I-DNA, O, O, O, B-DNA, I-DNA, O, O, B-protein, I-protein, I-protein, I-protein, O]"
5,9145,"[Involvement, of, mitogen-activated, protein, kinase, pathways, in, interleukin-8, production, by, human, monocytes, and, polymorphonuclear, cells, stimulated, with, lipopolysaccharide, or, Mycoplasma, fermentans, membrane, lipoproteins, .]","[O, O, B-protein, I-protein, I-protein, O, O, B-protein, O, O, B-cell_type, I-cell_type, O, B-cell_type, I-cell_type, O, O, O, O, B-protein, I-protein, I-protein, I-protein, O]"
6,6718,"[TNF, is, one, of, the, cytokines, secreted, by, the, cells, of, the, immune, system, .]","[B-protein, O, O, O, O, B-protein, O, O, O, O, O, O, O, O, O]"
7,35742,"[A, small, 95-bp, fragment, of, the, TCR, alpha, control, region, that, contains, the, TCF-1, alpha, binding, site, juxtaposed, between, a, cAMP-response, element, (, the, CRE, or, T, alpha, 1, motif, ), and, the, binding, site, for, a, distinct, lymphoid-specific, protein, (, TCF-2, alpha, ), behaved, as, a, potent, T-cell-specific, enhancer, in, vivo, .]","[O, O, O, O, O, O, B-DNA, I-DNA, I-DNA, I-DNA, O, O, O, B-DNA, I-DNA, I-DNA, I-DNA, O, O, O, B-DNA, I-DNA, O, O, B-DNA, O, B-DNA, I-DNA, I-DNA, I-DNA, O, O, O, B-DNA, I-DNA, O, O, O, B-protein, I-protein, O, B-protein, I-protein, O, O, O, O, O, B-DNA, I-DNA, O, O, O]"
8,5698,"[Furthermore, ,, similar, levels, of, NF, kappa, B, were, found, in, the, cytosols, of, unactivated, T, cells, from, both, young, and, elderly, donors, suggesting, that, precursor, levels, of, NF, kappa, B, remain, unaltered, during, aging, .]","[O, O, O, O, O, B-protein, I-protein, I-protein, O, O, O, O, O, O, O, B-cell_type, I-cell_type, O, O, O, O, O, O, O, O, O, O, O, B-protein, I-protein, I-protein, O, O, O, O, O]"
9,27652,"[NF-kappaB, binding, activity, correlated, with, the, degree, of, albuminuria, (, r, =, 0.316, ), and, with, thrombomodulin, plasma, concentrations, (, r, =, 0.33, ), ,, indicative, for, albuminuria, associated, endothelial, dysfunction, .]","[B-protein, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-protein, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pre0ed vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pre0ed` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pre0ing this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [164]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

loading configuration file config.json from cache at /Users/maxhager/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at /Users/maxhager/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/vocab.txt
l

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [165]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on one sentence:

In [166]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 8667, 117, 1142, 1110, 1141, 5650, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [167]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [101, 8667, 117, 1142, 1110, 1141, 5650, 3325, 1154, 1734, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pre0ed with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [168]:
example = datasets["train"][4]
print(example["tokens"])

['Our', 'data', 'suggest', 'that', 'lipoxygenase', 'metabolites', 'activate', 'ROI', 'formation', 'which', 'then', 'induce', 'IL-2', 'expression', 'via', 'NF-kappa', 'B', 'activation', '.']


In [169]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'Our', 'data', 'suggest', 'that', 'lip', '##ox', '##y', '##gen', '##ase', 'meta', '##bol', '##ites', 'activate', 'R', '##O', '##I', 'formation', 'which', 'then', 'induce', 'IL', '-', '2', 'expression', 'via', 'N', '##F', '-', 'ka', '##ppa', 'B', 'activation', '.', '[SEP]']


Here the words "Zwingmann" and "sheepmeat" have been split in three subtokens.

This means that we need to do some processing on our labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain, first because some special tokens might be added (we can a `[CLS]` and a `[SEP]` above) and then because of those possible splits of words in multiple tokens:

In [170]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(19, 35)

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [171]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 4, 4, 4, 4, 5, 5, 5, 6, 7, 7, 7, 8, 9, 10, 11, 12, 12, 12, 13, 14, 15, 15, 15, 15, 15, 16, 17, 18, None]


As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [172]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

35 35


Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [173]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [174]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [175]:
tokenize_and_align_labels(datasets["train"][:5])

{'input_ids': [[101, 15393, 118, 123, 5565, 2838, 1105, 151, 2271, 118, 24181, 13059, 139, 14915, 1194, 2891, 24606, 5315, 26844, 7621, 1707, 1118, 126, 118, 4764, 10649, 1183, 4915, 6530, 119, 102], [101, 2173, 16617, 1104, 1103, 2891, 24606, 2473, 10814, 2790, 170, 1558, 2616, 4060, 26658, 4344, 1111, 157, 2765, 14915, 3694, 1107, 9927, 1707, 1104, 9455, 1513, 17041, 1179, 118, 123, 113, 15393, 118, 123, 114, 1105, 2765, 23766, 119, 102], [101, 1130, 2425, 157, 181, 25698, 26431, 1195, 1437, 1115, 2891, 24606, 181, 13499, 2116, 4501, 1106, 1103, 6099, 1107, 4487, 18091, 3855, 1104, 26844, 7621, 9533, 1116, 113, 155, 2346, 2240, 1116, 114, 1134, 1132, 2320, 1111, 2891, 24606, 118, 22060, 14915, 1104, 1103, 151, 2271, 118, 24181, 13059, 139, 120, 2891, 24606, 118, 1231, 20080, 4199, 2109, 2703, 1105, 15393, 118, 123, 2838, 119, 102], [101, 9352, 2042, 1891, 1104, 1103, 2891, 24606, 16085, 11019, 26996, 2007, 1108, 1276, 1106, 8803, 4592, 189, 12577, 2155, 2042, 24779, 3246, 117, 1723, 

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our 0ing, validation and testing data will be preprocessed in one single command.

In [176]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /Users/maxhager/.cache/huggingface/datasets/jnlpba/jnlpba/1.0.0/3062f220823930cffde7976b694aa67bac3b06c322a02ced92d3761519810ce4/cache-f3127b525b7219d0.arrow
Loading cached processed dataset at /Users/maxhager/.cache/huggingface/datasets/jnlpba/jnlpba/1.0.0/3062f220823930cffde7976b694aa67bac3b06c322a02ced92d3761519810ce4/cache-ab74282edb9594ea.arrow


Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pre0ed model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pre0ed` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [177]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

loading configuration file config.json from cache at /Users/maxhager/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LABEL_9": 9
  },
  "layer_

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pre0 the model on a masked language modeling objective and replacing it with a new head for which we don't have pre0ed weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `0er`, we will need to define three more things. The most important is the [`0ingArguments`](https://huggingface.co/transformers/main_classes/0er.html#transformers.0ingArguments), which is a class that contains all the attributes to customize the 0ing. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [178]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for 0ing, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during 0ing. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-ner"` or `"huggingface/bert-finetuned-ner"`).

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [179]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `0er` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [180]:
metric = load_metric("seqeval")

This metric takes list of labels for the predictions and references:

In [181]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'protein': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `0er.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [182]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `0er`:

In [183]:
Trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `0` method:

In [184]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 37094
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6957
  Number of trainable parameters = 107728139


  0%|          | 0/6957 [00:00<?, ?it/s]

KeyboardInterrupt: 

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [ ]:
trainer.evaluate()

{'eval_loss': 0.05934586375951767,
 'eval_precision': 0.9292672127518264,
 'eval_recall': 0.9391430808815304,
 'eval_f1': 0.9341790463472988,
 'eval_accuracy': 0.9842565968195466,
 'epoch': 3.0}

To get the precision/recall/f1 computed for each category now that we have finished 0ing, we can apply the same function as before on the result of the `predict` method:

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'LOC': {'precision': 0.949718574108818,
  'recall': 0.966768525592055,
  'f1': 0.9581677077418134,
  'number': 2618},
 'MISC': {'precision': 0.8132387706855791,
  'recall': 0.8383428107229894,
  'f1': 0.8255999999999999,
  'number': 1231},
 'ORG': {'precision': 0.9055232558139535,
  'recall': 0.9090466926070039,
  'f1': 0.9072815533980583,
  'number': 2056},
 'PER': {'precision': 0.9759552042160737,
  'recall': 0.9765985497692815,
  'f1': 0.9762767710049424,
  'number': 3034},
 'overall_precision': 0.9292672127518264,
 'overall_recall': 0.9391430808815304,
 'overall_f1': 0.9341790463472988,
 'overall_accuracy': 0.9842565968195466}

You can now upload the result of the 0ing to the Hub, just execute this instruction:

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pre0ed("sgugger/my-awesome-model")
```